# Classification de texte en deep learning (LSTM et convolution) 

## But de la tâche 

A partir d'un dataset d'articles PUBMED, le but est de classifier les articles dans des catégories thématiques en fonction de leur titre. 

Après une phase de préprocessing du texte, nous entrainerons un modèle à base de convolutions, puis un modèle à base de réseau de neurones récurrents (LSTM) 

## Cloner le repo https://github.com/aneuraz/intro-keras.git

In [2]:
!git clone https://github.com/aneuraz/intro-keras.git

Cloning into 'intro-keras'...
remote: Enumerating objects: 18, done.
remote: Counting objects: 100% (18/18), done.
remote: Compressing objects: 100% (16/16), done.
remote: Total 18 (delta 5), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (18/18), done.


## Import des libraries

In [3]:
%tensorflow_version 2.x
import json 
import tensorflow as tf
import numpy as np

## Chargement des données

Toutes les données chargées se situent dans le répertoire `/content/`.
Les données sont dans un fichier JSON.

In [4]:
with open('/content/intro-keras/ai_pub_samp.json','r') as f:
  data = json.load(f)

In [5]:
data[0]

{'Cat_2013': 'C',
 'Cat_2014': 'C',
 'Cat_2015': 'C',
 'Cat_2016': 'C',
 'Cat_2017': 'B',
 'Disciplines': ['XQ'],
 'ESSN': '1873-3557',
 'IF_2013': '2.129',
 'IF_2014': '2.353',
 'IF_2015': '2.653',
 'IF_2016': '2.536',
 'IF_2017': '2.88',
 'ISSN': '1386-1425',
 'ISSN_online': '1873-3557',
 'ISSN_print': '1386-1425',
 'IsoAbbr': 'Spectrochim Acta A Mol Biomol Spectrosc',
 'JrId': 20555,
 'MedAbbr': 'Spectrochim Acta A Mol Biomol Spectrosc',
 'NLMid': '9602533',
 'Titre': 'Spectrochim Acta A Mol Biomol Spectrosc',
 'abstract': 'In this research, ZnO nanoparticle loaded on activated carbon (ZnO-NPs-AC) was synthesized simply by a low cost and nontoxic procedure. The characterization and identification have been completed by different techniques such as SEM and XRD analysis. A three layer artificial neural network (ANN) model is applicable for accurate prediction of dye removal percentage from aqueous solution by ZnO-NRs-AC following conduction of 270 experimental data. The network was tr

## TODO: Extraire les titres et les catégories

In [6]:
# mettre le titre en minuscule dans la variable X
X = [ x['title'].lower() for x in data ] 

# mettre la catégorie (1e élément de la liste) dans la variable Y
Y = [ y['categories'][0] for y in data ] 

## TODO: Calculer la longueur maximale des titres dans le dataset

In [7]:
# longueur maximale des titres, variable max_len
max_len = len(max(X, key=len))

## TODO: Diviser le dataset en train (X_train, Y_train) et test (X_test, Y_test)

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size = 0.2, random_state = 21)

# X_train, Y_train
X_train = X[:8000]
Y_train = Y[:8000]

# X_test, Y_test
X_test = X[8000:]
Y_test = Y[8000:]


## Transformer la variable Y en vecteur numerique

["Cat 1", "Cat 2"] -> [0, 1]

In [18]:
cat_to_id = {'<UNK>': 0}


for t in Y_train:
  if not t in cat_to_id.keys():
    cat_to_id[t] = len(cat_to_id)

id_to_cat = {v: k for k,v in cat_to_id.items()}


# creer un mapping cat_2_id


# creer un reverse mapping id_2_cat

# calculer la taille du vocabulaire cat_vocab

num_cat = len(cat_to_id)


In [10]:
# preprocesser les X_train et X_test en X_train_id et X_test_id

def preprocess_Y(Y, cat_to_id):
  res = []
  for ex in Y:
    if ex not in cat_to_id.keys():
      res.append(cat_to_id['<UNK>'])
    else:
      res.append(cat_to_id[ex])
    
  return np.array(res)

Y_train_id = preprocess_Y(Y_train, cat_to_id)
Y_test_id = preprocess_Y(Y_test, cat_to_id)

## Tokenizer les titres

Pour cela vous pouvez utiliser la fonction `Tokenizer` de keras

Le but est de transformer les textes en un vecteur numérique

texte -> liste de tokens -> vecteur numérique

"Miaou le chat" -> ["Miaou", "le", chat"] -> [1, 2, 3]

In [11]:
# Créer le tokenizer
vocab_size = 10000
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words = vocab_size)


In [12]:
# Entrainer le tokenizer sur le train set 
tokenizer.fit_on_texts(X_train)

In [37]:
# Transformer les textes en vecteurs numeriques à l'aide du tokenizer
X_train_seq = tokenizer.texts_to_sequences(X_train)
X_test_seq = tokenizer.texts_to_sequences(X_test)

X_train[0]
#X_train_seq[0]

'artificial neural network (ann) method for modeling of sunset yellow dye adsorption using zinc oxide nanorods loaded on activated carbon: kinetic and isotherm study.'

## Faire un padding des sequences obtenues pour qu'elles aient toutes la même taille (cf la fonction `pad_sequences`)

[1, 2, 3]       -> [0, 0, 1, 2 ,3]

[4, 5, 6, 7, 8] -> [4, 5, 6, 7, 8]

In [14]:
max_len = max([len(x) for x in X_train_seq])
# Padding des sequences 
X_train_pad = tf.keras.preprocessing.sequence.pad_sequences(X_train_seq, maxlen=max_len, truncating='post')
X_test_pad = tf.keras.preprocessing.sequence.pad_sequences(X_test_seq, maxlen=max_len, truncating='post')

In [15]:
X_train_pad[0:5]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,   21,    9,   16,  945,   48,    4,   79,
           1, 5677, 1973, 2247, 2614,    7, 2248, 2615, 5678, 3995,   13,
        1586,  946, 1974,    2, 5679,   37],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,  462,    1,   74,  124,   48,    3,  615,
         616, 3135,   10,    5,  693,   48],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,  135,   14,    1,  589,  143,
         327,    3, 3136, 1337,   64, 2616],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,  802, 1587,  

In [16]:
X_test_pad[0:5]

array([[   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,   23,   18, 3459, 9708,
         197,  117,    3,    6,   85, 1354],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,   72,   89,    3,  911,  818,    3,    5, 1288,    1,
         116,   14,    1,  255, 9110,  412],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,   46,   18,   23,  262,
         142, 1309,    3,   19,  823,   33],
       [   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
           0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
         368,    2,  561,  384,  701,    1, 3827,  

In [17]:
Y_test_id[0:100]

array([29, 29, 10,  5,  5,  5,  6, 21, 21, 44, 21, 10, 20, 43,  6, 14, 55,
        2,  7,  2, 12, 69, 10, 55, 10, 29, 10, 43, 30, 14, 78, 10,  5, 10,
        3, 12,  5, 64, 48, 55, 11,  5,  5,  5,  5, 21, 85, 25, 10,  2, 11,
       18, 10,  5, 20, 55,  6,  5, 10,  1, 45,  6,  2, 20, 12,  8,  2,  5,
       18, 48,  1, 10, 26, 12, 70, 58, 12,  5, 12, 14,  2,  5,  5, 10, 21,
        5, 12, 27,  7, 21,  5, 10,  5, 20,  5, 49, 45, 13, 16, 20])

# Réseau de convolution pour la classification de texte

Les réseaux convolutionnels peuvent également être utiliser pour le texte et notamment pour la classification de texte. Ici nous allons construire un CNN sur le même modèle que pour les images avec quelques petites spécificités. 

Comme le texte est une séquence de mots, il s'agit d'une séquence en 1 dimension. Nous appliquerons donc une convolution en 1D. 

Pour traiter du texte, la première couche de notre réseau va être constituée par une couche d'embedding. 

Pour rappel, le word embedding consiste à projeter les tokens dans un espace vectoriel qui va minimiser la distance entre les tokens qui sont utilisés dans des contextes similaires (et qui ont un sens proche ? )

![Texte alternatif…](https://www.ibm.com/blogs/research/wp-content/uploads/2018/10/WMEFig1.png)

Les embeddings peuvent être calculés de diverses façons. Par exemple word2vec, un des plus célèbres, se base sur 2 algorithmes frères Skip-gram et CBOW

![Texte alternatif…](https://pathmind.com/images/wiki/word2vec_diagrams.png)

Pour information, il existe aujourd'hui des algorithmes plus performants que word2vec comme [Fasttext](https://fasttext.cc) qui prend en compte des informations de sous-mots ou la famille des embeddings contextuels comme [ELMo](https://allennlp.org/elmo) ou [BERT](https://arxiv.org/abs/1810.04805) qui prennent en compte le contexte d'utilisation du mot pour calculer son vecteur.

> Bloc en retrait



In [19]:
embed_dim= 128
dropout1 = 0.2
conv_filters= 32
conv_kernel = 2
maxpool_size = 2
dense_size = 128
batch_size = 128
epochs = 10

model_cnn = tf.keras.models.Sequential()

# Créer le modèle avec au minimum
# Embedding 
model_cnn.add(tf.keras.layers.Embedding(vocab_size, 
                                        embed_dim, 
                                        input_length= max_len))

# Dropout
model_cnn.add(tf.keras.layers.Dropout(dropout1))

# Convolution
model_cnn.add(tf.keras.layers.Conv1D(conv_filters, conv_kernel, 
                                     padding='valid', 
                                     strides= 1, 
                                     activation='relu'))
# Maxpooling
model_cnn.add(tf.keras.layers.MaxPooling1D(maxpool_size))
model_cnn.add(tf.keras.layers.Flatten())

# Dense
model_cnn.add(tf.keras.layers.Dense(dense_size,  activation='relu'))

# Activation
# Classifieur (Dense + activation softmax)
model_cnn.add(tf.keras.layers.Dense(num_cat))
model_cnn.add(tf.keras.layers.Activation('softmax'))

# Compiler le modèle 
model_cnn.compile(loss='sparse_categorical_crossentropy', 
                  optimizer='adam', 
                  metrics= ['accuracy'])

# Afficher le summary du modèle
print(model_cnn.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 39, 128)           1280000   
_________________________________________________________________
dropout (Dropout)            (None, 39, 128)           0         
_________________________________________________________________
conv1d (Conv1D)              (None, 38, 32)            8224      
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 19, 32)            0         
_________________________________________________________________
flatten (Flatten)            (None, 608)               0         
_________________________________________________________________
dense (Dense)                (None, 128)               77952     
_________________________________________________________________
dense_1 (Dense)              (None, 97)                1

In [20]:
# Fitter le modèle 
model_cnn.fit(X_train_pad, Y_train_id, batch_size = batch_size , epochs = epochs)

Epoch 1/10
63/63 [==============================] - 1s 15ms/step - loss: 3.6857 - accuracy: 0.1844
Epoch 2/10
63/63 [==============================] - 1s 15ms/step - loss: 3.3163 - accuracy: 0.2216
Epoch 3/10
63/63 [==============================] - 1s 15ms/step - loss: 2.9923 - accuracy: 0.2829
Epoch 4/10
63/63 [==============================] - 1s 15ms/step - loss: 2.5880 - accuracy: 0.3792
Epoch 5/10
63/63 [==============================] - 1s 15ms/step - loss: 2.1363 - accuracy: 0.4745
Epoch 6/10
63/63 [==============================] - 1s 14ms/step - loss: 1.7061 - accuracy: 0.5907
Epoch 7/10
63/63 [==============================] - 1s 15ms/step - loss: 1.3387 - accuracy: 0.6771
Epoch 8/10
63/63 [==============================] - 1s 15ms/step - loss: 1.0453 - accuracy: 0.7554
Epoch 9/10
63/63 [==============================] - 1s 15ms/step - loss: 0.7999 - accuracy: 0.8154
Epoch 10/10
63/63 [==============================] - 1s 15ms/step - loss: 0.6134 - accuracy: 0.8655


In [21]:
# Evaluer le modèle
model_cnn.evaluate(X_test_pad, Y_test_id)

63/63 [==============================] - 0s 3ms/step - loss: 3.8217 - accuracy: 0.3120


[3.8216774463653564, 0.31200000643730164]

# LSTM pour la classification de texte

Il est également possible d'utiliser un autre type de réseau de neurones pour effectuer ce genre de tâches: les réseaux de neurones récurrents ou RNN.

Les RNN sont conçus pour gérer les séquences. Le réseau prend les tokens un par un et calcule une représentation de la séquence à chaque pas qui tiens compte de tous les pas précédents 

![Texte alternatif…](https://upload.wikimedia.org/wikipedia/commons/thumb/b/b5/Recurrent_neural_network_unfold.svg/450px-Recurrent_neural_network_unfold.svg.png)


Il existe différents types de RNN. Ici nous utiliserons les Long Short-Term Memory (LSTM) qui permettent d'améliorer les performances sur des séquences longues avec une série de "gates". 

![Texte alternatif…](http://dprogrammer.org/wp-content/uploads/2019/04/RNN-vs-LSTM-vs-GRU-1200x361.png)

In [28]:
lstm_size= 512
dropout2= 0.2

# Créer un réseau à base de LSTM avec au minimum:
model_lstm = tf.keras.models.Sequential()
# Embedding
model_lstm.add(tf.keras.layers.Embedding(vocab_size, 
                                        embed_dim, 
                                        input_length= max_len))
# Dropout
model_lstm.add(tf.keras.layers.Dropout(dropout1))
# LSTM
model_lstm.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_size //2)))
# Dropout
model_lstm.add(tf.keras.layers.Dropout(dropout2))
# Classifieur (Dense + activation softmax)
model_lstm.add(tf.keras.layers.Dense(num_cat))
model_lstm.add(tf.keras.layers.Activation('softmax'))



# Compiler le modèle 
model_lstm.compile(loss='sparse_categorical_crossentropy', 
                  optimizer='adam', 
                  metrics= ['accuracy'])

# Afficher le summary du modèle
print(model_lstm.summary())



Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 39, 128)           1280000   
_________________________________________________________________
dropout_5 (Dropout)          (None, 39, 128)           0         
_________________________________________________________________
bidirectional_2 (Bidirection (None, 512)               788480    
_________________________________________________________________
dropout_6 (Dropout)          (None, 512)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 97)                49761     
_________________________________________________________________
activation_3 (Activation)    (None, 97)                0         
Total params: 2,118,241
Trainable params: 2,118,241
Non-trainable params: 0
____________________________________________

In [29]:
# Fitter le modèle
model_lstm.fit(X_train_pad, Y_train_id, batch_size = batch_size , epochs = epochs)

Epoch 1/10
63/63 [==============================] - 2s 28ms/step - loss: 3.6067 - accuracy: 0.1831
Epoch 2/10
63/63 [==============================] - 2s 26ms/step - loss: 3.2601 - accuracy: 0.2375
Epoch 3/10
63/63 [==============================] - 2s 26ms/step - loss: 2.7702 - accuracy: 0.3173
Epoch 4/10
63/63 [==============================] - 2s 26ms/step - loss: 2.3720 - accuracy: 0.3915
Epoch 5/10
63/63 [==============================] - 2s 26ms/step - loss: 2.0231 - accuracy: 0.4773
Epoch 6/10
63/63 [==============================] - 2s 25ms/step - loss: 1.7230 - accuracy: 0.5533
Epoch 7/10
63/63 [==============================] - 2s 25ms/step - loss: 1.4743 - accuracy: 0.6093
Epoch 8/10
63/63 [==============================] - 2s 24ms/step - loss: 1.2363 - accuracy: 0.6662
Epoch 9/10
63/63 [==============================] - 2s 25ms/step - loss: 1.0288 - accuracy: 0.7289
Epoch 10/10
63/63 [==============================] - 2s 25ms/step - loss: 0.8681 - accuracy: 0.7721


In [30]:
# Evaluer le modèle
model_lstm.evaluate(X_test_pad, Y_test_id)

63/63 [==============================] - 0s 6ms/step - loss: 3.5024 - accuracy: 0.3250


[3.5024211406707764, 0.32499998807907104]

# Utiliser les embeddings pré-entrainés

Pour améliorer la qualité de la représentation des mots, il est possible d'entrainer les embeddings sur de larges corpus de textes non annotés (typiquement Wikipedia). Ces modèles sont souvent disponibles en ligne et il est possible de les télécharger. Ici nous allons utiliser des embeddings [Glove](https://nlp.stanford.edu/projects/glove/) de taille 50d (pour des raisons techniques mais il vaut mieux utiliser des dimensions plus importantes entre 100 et 300) 

In [31]:
# Fonction permettant de charger un embedding 

import numpy as np
import re
from nltk.tokenize import word_tokenize
import nltk
nltk.download('punkt')

def load_glove_embeddings(fp, embedding_dim, include_empty_char=True):
    """
    Loads pre-trained word embeddings (GloVe embeddings)
        Inputs: - fp: filepath of pre-trained glove embeddings
                - embedding_dim: dimension of each vector embedding
                - generate_matrix: whether to generate an embedding matrix
        Outputs:
                - word2coefs: Dictionary. Word to its corresponding coefficients
                - word2index: Dictionary. Word to word-index
                - embedding_matrix: Embedding matrix for Keras Embedding layer
    """
    # First, build the "word2coefs" and "word2index"
    word2coefs = {} # word to its corresponding coefficients
    word2index = {} # word to word-index
    with open(fp) as f:
        for idx, line in enumerate(f):
            try:
                data = [x.strip().lower() for x in line.split()]
                word = data[0]
                coefs = np.asarray(data[1:embedding_dim+1], dtype='float32')
                word2coefs[word] = coefs
                if word not in word2index:
                    word2index[word] = len(word2index)
            except Exception as e:
                print('Exception occurred in `load_glove_embeddings`:', e)
                continue
        # End of for loop.
    # End of with open
    if include_empty_char:
        word2index[''] = len(word2index)
    # Second, build the "embedding_matrix"
    # Words not found in embedding index will be all-zeros. Hence, the "+1".
    vocab_size = len(word2coefs)+1 if include_empty_char else len(word2coefs)
    embedding_matrix = np.zeros((vocab_size, embedding_dim))
    for word, idx in word2index.items():
        embedding_vec = word2coefs.get(word)
        if embedding_vec is not None and embedding_vec.shape[0]==embedding_dim:
            embedding_matrix[idx] = np.asarray(embedding_vec)
    # return word2coefs, word2index, embedding_matrix
    return word2index, np.asarray(embedding_matrix)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [32]:
# Télécharger les embeddings

!wget https://github.com/kmr0877/IMDB-Sentiment-Classification-CBOW-Model/raw/master/glove.6B.50d.txt.gz
!gunzip /content/glove.6B.50d.txt.gz

--2020-12-09 20:45:34--  https://github.com/kmr0877/IMDB-Sentiment-Classification-CBOW-Model/raw/master/glove.6B.50d.txt.gz
Resolving github.com (github.com)... 192.30.255.113
Connecting to github.com (github.com)|192.30.255.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/kmr0877/IMDB-Sentiment-Classification-CBOW-Model/master/glove.6B.50d.txt.gz [following]
--2020-12-09 20:45:34--  https://raw.githubusercontent.com/kmr0877/IMDB-Sentiment-Classification-CBOW-Model/master/glove.6B.50d.txt.gz
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 69182520 (66M) [application/octet-stream]
Saving to: ‘glove.6B.50d.txt.gz’

glove.6B.50d.txt.gz 100%[===================>]  65.98M   162MB/s    in 0.4s    

202

In [33]:
# Charger les embeddings à l'aide de la fonction load_glove_embeddings
word2index, embedding_matrix = load_glove_embeddings('glove.6B.50d.txt', 50)

In [54]:
# ecrire une fonction de tokenization custom pour preprocesser les textes
import re 
def custom_tokenizer(input, word2index):

  # Split words
  res = []
  for t in input:
    words = re.findall(r'\w+', t)
    index = []
    for i in words:
      if i in word2index.keys():
        index.append(word2index[i])
    res.append(index)

  return res

# Encoder les textes avec la fonction custom
X_train_seq = custom_tokenizer(X_train, word2index)
X_test_seq = custom_tokenizer(X_test, word2index)


In [55]:
# Padding des sequences
max_len = max([len(x) for x in X_train_seq])
# Padding des sequences 
X_train_pad = tf.keras.preprocessing.sequence.pad_sequences(X_train_seq, maxlen=max_len, truncating='post')
X_test_pad = tf.keras.preprocessing.sequence.pad_sequences(X_test_seq, maxlen=max_len, truncating='post')

In [69]:
# Créer un modèle en chargeant les poids des embeddings dans le layer Embedding
lstm_size= 512
dropout2= 0.2

embed_dim= 50
dropout1 = 0.2
conv_filters= 32
conv_kernel = 2
maxpool_size = 2
dense_size = 128

# Créer un réseau à base de LSTM avec au minimum:
model_lstm = tf.keras.models.Sequential()
# Embedding
model_lstm.add(tf.keras.layers.Embedding(400001, 
                                         embed_dim, 
                                         weights=[embedding_matrix], 
                                         input_length=max_len, 
                                         trainable=False))
#model_lstm.add(tf.keras.layers.Embedding(vocab_size, 
#                                        embed_dim, 
#                                        input_length= max_len))
# Dropout
model_lstm.add(tf.keras.layers.Dropout(dropout1))
# LSTM
model_lstm.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(lstm_size //2)))
# Dropout
model_lstm.add(tf.keras.layers.Dropout(dropout2))
# Classifieur (Dense + activation softmax)
model_lstm.add(tf.keras.layers.Dense(num_cat))
model_lstm.add(tf.keras.layers.Activation('softmax'))



# Compiler le modèle 
model_lstm.compile(loss='sparse_categorical_crossentropy', 
                  optimizer='adam', 
                  metrics= ['accuracy'])

# Afficher le summary du modèle
print(model_lstm.summary())


Model: "sequential_9"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (None, 39, 50)            20000050  
_________________________________________________________________
dropout_15 (Dropout)         (None, 39, 50)            0         
_________________________________________________________________
bidirectional_7 (Bidirection (None, 512)               628736    
_________________________________________________________________
dropout_16 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 97)                49761     
_________________________________________________________________
activation_8 (Activation)    (None, 97)                0         
Total params: 20,678,547
Trainable params: 678,497
Non-trainable params: 20,000,050
____________________________________

In [70]:
# Fitter le modèle
batch_size = 256
epochs = 100
model_lstm.fit(X_train_pad, Y_train_id, batch_size = batch_size , epochs = epochs)


Epoch 1/100
63/63 [==============================] - 1s 15ms/step - loss: 3.4030 - accuracy: 0.2122
Epoch 2/100
63/63 [==============================] - 1s 13ms/step - loss: 2.8736 - accuracy: 0.2864
Epoch 3/100
63/63 [==============================] - 1s 13ms/step - loss: 2.6909 - accuracy: 0.3090
Epoch 4/100
63/63 [==============================] - 1s 13ms/step - loss: 2.5670 - accuracy: 0.3293
Epoch 5/100
63/63 [==============================] - 1s 13ms/step - loss: 2.4905 - accuracy: 0.3438
Epoch 6/100
63/63 [==============================] - 1s 13ms/step - loss: 2.4197 - accuracy: 0.3531
Epoch 7/100
63/63 [==============================] - 1s 12ms/step - loss: 2.3449 - accuracy: 0.3661
Epoch 8/100
63/63 [==============================] - 1s 13ms/step - loss: 2.2688 - accuracy: 0.3809
Epoch 9/100
63/63 [==============================] - 1s 13ms/step - loss: 2.2165 - accuracy: 0.3931
Epoch 10/100
63/63 [==============================] - 1s 13ms/step - loss: 2.1489 - accuracy: 0.4021

In [72]:
# evaluer le modèle
model_lstm.evaluate(X_test_pad, Y_test_id)

63/63 [==============================] - 0s 6ms/step - loss: 3.9401 - accuracy: 0.3345


[3.9401023387908936, 0.3345000147819519]